<a href="https://colab.research.google.com/github/raghurammanikanta/WORK-ON-TELUGU-DATA/blob/main/3ET_on_emotion_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
  Attempting uninstall: sphinx
    Found existing installation: Sphinx 5.0.2
    Uninstalling Sphinx-5.0.2:
      Successfully uninstalled Sphinx-5.0.2


# Import Libraries and Load Dataset

In [ ]:
import pandas as pd
import re
import numpy as np
from gensim.models import FastText, Word2Vec
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.ensemble import RandomForestClassifier
from indicnlp.tokenize import indic_tokenize



In [ ]:
# # Load and preprocess dataset
df = pd.read_csv('/content/drive/MyDrive/emotion_filtered.csv')
# df.head()
# df['topic'] = df['topic'].apply(lambda x: x if x in ['entertainment', 'nation'] else 'others')
# c = df['topic'].value_counts()
# c


In [ ]:
df.head()
y = df['Emotion']

In [ ]:
df['Emotion'].value_counts()

,count
Emotion,
no,38193
sad,7658
happy,6073
angry,595
fear,189


# **Text Preprocessing**

In [ ]:
def pre_pro(text):
    text = re.sub(r'[^ఁ-౿ ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Sentence'] = df['Sentence'].apply(pre_pro)
print("\nPreprocessed Text:")
print(df.head())


Preprocessed Text:
                                            Sentence Emotion Tokens  \
0  డ్రెస్సింగ్ విషయంలో సెలబ్రిటీలను బయటివాళ్లూ అన...      no    NaN   
1  అయితే ప్రభుదాస్ వైష్ణాని అనే వ్యక్తి కస్టడీలో ...     sad    NaN   
2  విజయ్ ఆంటోని యాక్షన్కింగ్ అర్జున్ కలిసి నటిస్త...      no    NaN   
3  ఆయన ఎన్నికల్లో టీడీపీ సిట్టింగ్ ఎమ్మెల్యే పిల్...   happy    NaN   
4  రెండోసారి ప్రధాని అయిన నరేంద్రమోదీకి ఏపీలో విజ...   happy    NaN   

   Sentence_Length  
0              166  
1               60  
2               69  
3              110  
4              108  


FastText Model Training and Embedding Extraction

In [ ]:
df['Sentence'] = df['Sentence'].apply(lambda x: x.split() if isinstance(x, str) else x)
embedding_size = 100 ##embedding_size is the size of the embedding vector.
window_size = 5 #window_size is the size of the number of words
min_word = 6 #min_word, which specifies the minimum frequency of a word
down_sampling = 1e-2 #most frequently occurring word will be down-sampled by a number specified by the down_sampling attribute


ft_model = FastText(sentences = df['Sentence'],vector_size=embedding_size,window=window_size,min_count=min_word,sample=down_sampling,sg=1,epochs=10)
#sg defines if the parameter is skpgram or cbow

print(ft_model.wv)

# Extract word embeddings
X = []
for sentence in df['Sentence']:
    word_embeddings = [ft_model.wv[word] for word in sentence if word in ft_model.wv]
    if word_embeddings:
        sentence_embedding = np.mean(word_embeddings, axis=0)
        X.append(sentence_embedding)
    else:
        X.append(np.zeros(embedding_size))

X = np.array(X)


FastTextKeyedVectors<vector_size=100, 13218 keys>


TF-IDF Embedding Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the tokenized words back into sentences
df['text_joined'] = df['Sentence'].apply(lambda x: ' '.join(x))

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_joined'])

# `tfidf_matrix` now contains your TF-IDF embeddings
print(tfidf_matrix)

  (0, 3380)	0.218641738202669
  (0, 3034)	0.3025260131888343
  (0, 3012)	0.14658565233136103
  (0, 2322)	0.3377062989512917
  (0, 49)	0.14259880864478994
  (0, 858)	0.1846412600475282
  (0, 1154)	0.22397061918723815
  (0, 3231)	0.20748472888165437
  (0, 47)	0.19217869993760123
  (0, 856)	0.4934973688859177
  (0, 1256)	0.23378976704089166
  (0, 2561)	0.18473137373350074
  (0, 80)	0.2098264053218124
  (0, 3020)	0.2325433017783746
  (0, 272)	0.25340034299567216
  (0, 1167)	0.21431391671521746
  (1, 49)	0.242766176376879
  (1, 2561)	0.3144944174803156
  (1, 1167)	0.36485697601378364
  (1, 93)	0.31019263206416825
  (1, 2875)	0.3558482205438009
  (1, 942)	0.5045197412112887
  (1, 1389)	0.4812999456806044
  (2, 1285)	0.4360305687264848
  (2, 3365)	0.41250900540748814
  :	:
  (52705, 96)	0.2790054036537253
  (52705, 2008)	0.2196103921848573
  (52705, 2877)	0.22288229818091354
  (52705, 270)	0.22379877853466723
  (52705, 2127)	0.31853398634578245
  (52705, 418)	0.23150566663839264
  (52705, 299

In [ ]:
TY = tfidf_matrix

Word2Vec Model Training and Embedding Extraction

In [ ]:

from indicnlp.tokenize import indic_tokenize

# Assuming df['body'] contains your Telugu text data
# df = pd.read_csv('/content/TELUGU_NEWS_TEST.csv')  # Load your Telugu data

# Tokenize Telugu sentences
sentences = [indic_tokenize.trivial_tokenize("".join(sentence).lower()) for sentence in df['Sentence']]

# CBOW model
cbow_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Skip-gram model
sg_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Save the models
cbow_model.save("telugu_word2vec_cbow.model")
sg_model.save("telugu_word2vec_sg.model")
w2v_model = Word2Vec.load("telugu_word2vec_cbow.model")

def get_document_vector(doc):
    # Check if doc is a list and join it into a string if necessary
    if isinstance(doc, list):
        doc = " ".join(doc)
    words = doc.split()
    word_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(word_vecs) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(word_vecs, axis=0)

# Create document vectors
X_w2v = np.array([get_document_vector(doc) for doc in df['Sentence']])
yw = df['Emotion']

 Train-Test Split

In [ ]:
# Split the data
WX_train, WX_test, Wy_train, Wy_test = train_test_split(X_w2v, yw, test_size=0.2, random_state=42)
TX_train, TX_test, Ty_train, Ty_test = train_test_split(TY, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


 SVM Model Training and Evaluation

In [ ]:
# SVM with TF-IDF
clf = SVC(kernel='linear')  # You can explore other kernels as well
clf.fit(TX_train, Ty_train)
Ty_pred = clf.predict(TX_test)
print(f'\nSVM with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# SVM with FastText
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'\nSVM with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# SVM with Word2Vec
svm_model = SVC(kernel='linear')
svm_model.fit(WX_train, Wy_train)
yw_pred = svm_model.predict(WX_test)
print(f'\nSVM with Word2Vec Accuracy: {accuracy_score(Wy_test, y_pred)}')
print(classification_report(Wy_test, yw_pred))



SVM with TF-IDF Accuracy: 0.7374312274710682
              precision    recall  f1-score   support

       angry       0.56      0.07      0.12       130
        fear       0.73      0.38      0.50        29
       happy       0.53      0.06      0.10      1197
          no       0.74      0.98      0.85      7605
         sad       0.67      0.14      0.23      1581

    accuracy                           0.74     10542
   macro avg       0.65      0.33      0.36     10542
weighted avg       0.70      0.74      0.66     10542


SVM with FastText Accuracy: 0.7214001138303927


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.52      0.72      0.60     10542


SVM with Word2Vec Accuracy: 0.7214001138303927
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.52      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KNN Model Training and Evaluation

In [ ]:
# KNN with FastText
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)
y_pred = knn_classifier.predict(X_test)
print(f'\nKNN with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# KNN with TF-IDF
knn_classifier.fit(TX_train, Ty_train)
Ty_pred = knn_classifier.predict(TX_test)
print(f'\nKNN with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# KNN with Word2Vec
knn_classifier.fit(WX_train, Wy_train)
y_pred = knn_classifier.predict(WX_test)
print(f'\nKNN with Word2Vec Accuracy: {accuracy_score(Wy_test, y_pred)}')
print(classification_report(Wy_test, y_pred))



KNN with FastText Accuracy: 0.7481502561183836
              precision    recall  f1-score   support

       angry       0.18      0.20      0.19       130
        fear       0.13      0.10      0.12        29
       happy       0.45      0.37      0.40      1197
          no       0.81      0.89      0.85      7605
         sad       0.61      0.42      0.50      1581

    accuracy                           0.75     10542
   macro avg       0.44      0.40      0.41     10542
weighted avg       0.73      0.75      0.74     10542


KNN with TF-IDF Accuracy: 0.7403718459495352
              precision    recall  f1-score   support

       angry       0.41      0.17      0.24       130
        fear       0.26      0.28      0.27        29
       happy       0.42      0.31      0.36      1197
          no       0.78      0.92      0.85      7605
         sad       0.64      0.24      0.35      1581

    accuracy                           0.74     10542
   macro avg       0.50      0.38    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.52      0.72      0.60     10542



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logistic Regression Model Training and Evaluation

In [ ]:
# Logistic Regression with FastText
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, y_train)
y_pred = logreg_classifier.predict(X_test)
print(f'\nLogistic Regression with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Logistic Regression with TF-IDF
logreg_classifier.fit(TX_train, Ty_train)
Ty_pred = logreg_classifier.predict(TX_test)
print(f'\nLogistic Regression with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Logistic Regression with Word2Vec
logreg_classifier.fit(WX_train, Wy_train)
Wy_pred = logreg_classifier.predict(WX_test)
print(f'\nLogistic Regression with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Logistic Regression with FastText Accuracy: 0.7355340542591539


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.25      0.01      0.01       130
        fear       0.00      0.00      0.00        29
       happy       0.51      0.09      0.15      1197
          no       0.75      0.96      0.84      7605
         sad       0.58      0.22      0.31      1581

    accuracy                           0.74     10542
   macro avg       0.42      0.25      0.26     10542
weighted avg       0.69      0.74      0.67     10542



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Logistic Regression with TF-IDF Accuracy: 0.7472016695124265
              precision    recall  f1-score   support

       angry       0.59      0.08      0.14       130
        fear       1.00      0.17      0.29        29
       happy       0.57      0.12      0.19      1197
          no       0.76      0.97      0.85      7605
         sad       0.66      0.22      0.33      1581

    accuracy                           0.75     10542
   macro avg       0.72      0.31      0.36     10542
weighted avg       0.72      0.75      0.69     10542


Logistic Regression with Word2Vec Accuracy: 0.7214001138303927


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.52      0.72      0.60     10542



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Decision Tree Model Training and Evaluation

In [ ]:
# Decision Tree with FastText
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
y_pred = dt_classifier.predict(X_test)
print(f'\nDecision Tree with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Decision Tree with TF-IDF
dt_classifier.fit(TX_train, Ty_train)
Ty_pred = dt_classifier.predict(TX_test)
print(f'\nDecision Tree with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Decision Tree with Word2Vec
dt_classifier.fit(WX_train, Wy_train)
Wy_pred = dt_classifier.predict(WX_test)
print(f'\nDecision Tree with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))



Decision Tree with FastText Accuracy: 0.8890153671030165
              precision    recall  f1-score   support

       angry       0.72      0.75      0.74       130
        fear       0.68      0.86      0.76        29
       happy       0.77      0.82      0.79      1197
          no       0.94      0.92      0.93      7605
         sad       0.78      0.80      0.79      1581

    accuracy                           0.89     10542
   macro avg       0.78      0.83      0.80     10542
weighted avg       0.89      0.89      0.89     10542


Decision Tree with TF-IDF Accuracy: 0.8981217985202049
              precision    recall  f1-score   support

       angry       0.84      0.77      0.80       130
        fear       0.78      0.86      0.82        29
       happy       0.80      0.77      0.78      1197
          no       0.92      0.95      0.94      7605
         sad       0.84      0.77      0.81      1581

    accuracy                           0.90     10542
   macro avg     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.52      0.72      0.60     10542



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Naive Bayes Model Training and Evaluation

In [ ]:
# Naive Bayes with FastText
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_scaled, y_train)
y_pred = nb_classifier.predict(X_test_scaled)
print(f'\nNaive Bayes with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Naive Bayes with TF-IDF
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(TX_train)
X_test_scaled = scaler.transform(TX_test)
nb_classifier.fit(X_train_scaled, Ty_train)
Ty_pred = nb_classifier.predict(X_test_scaled)
print(f'\nNaive Bayes with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Naive Bayes with Word2Vec
scaler = MinMaxScaler()
WX_train_scaled = scaler.fit_transform(WX_train)
WX_test_scaled = scaler.transform(WX_test)
nb_classifier.fit(WX_train_scaled, Wy_train)
Wy_pred = nb_classifier.predict(WX_test_scaled)
print(f'\nNaive Bayes with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))



Naive Bayes with FastText Accuracy: 0.7214001138303927


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.52      0.72      0.60     10542


Naive Bayes with TF-IDF Accuracy: 0.7429330297856195


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       angry       0.43      0.02      0.04       130
        fear       0.00      0.00      0.00        29
       happy       0.71      0.06      0.12      1197
          no       0.74      0.99      0.85      7605
         sad       0.74      0.15      0.25      1581

    accuracy                           0.74     10542
   macro avg       0.53      0.24      0.25     10542
weighted avg       0.73      0.74      0.66     10542


Naive Bayes with Word2Vec Accuracy: 0.7214001138303927
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       130
        fear       0.00      0.00      0.00        29
       happy       0.00      0.00      0.00      1197
          no       0.72      1.00      0.84      7605
         sad       0.00      0.00      0.00      1581

    accuracy                           0.72     10542
   macro avg       0.14      0.20      0.17     10542
weighted avg       0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest Model Training and Evaluation

In [ ]:
# Random Forest with FastText
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
print(f'\nRandom Forest with FastText Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

# Random Forest with TF-IDF
rf_classifier.fit(TX_train, Ty_train)
Ty_pred = rf_classifier.predict(TX_test)
print(f'\nRandom Forest with TF-IDF Accuracy: {accuracy_score(Ty_test, Ty_pred)}')
print(classification_report(Ty_test, Ty_pred))

# Random Forest with Word2Vec
rf_classifier.fit(WX_train, Wy_train)
Wy_pred = rf_classifier.predict(WX_test)
print(f'\nRandom Forest with Word2Vec Accuracy: {accuracy_score(Wy_test, Wy_pred)}')
print(classification_report(Wy_test, Wy_pred))



Random Forest with FastText Accuracy: 0.9299943084803642
              precision    recall  f1-score   support

       angry       0.97      0.75      0.84       130
        fear       0.93      0.86      0.89        29
       happy       0.98      0.77      0.86      1197
          no       0.92      0.99      0.95      7605
         sad       0.98      0.76      0.85      1581

    accuracy                           0.93     10542
   macro avg       0.95      0.83      0.88     10542
weighted avg       0.93      0.93      0.93     10542


Random Forest with TF-IDF Accuracy: 0.9168089546575602
              precision    recall  f1-score   support

       angry       0.95      0.75      0.84       130
        fear       0.89      0.86      0.88        29
       happy       0.93      0.74      0.82      1197
          no       0.91      0.98      0.95      7605
         sad       0.94      0.75      0.84      1581

    accuracy                           0.92     10542
   macro avg     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
